## 1050. Actors and Directors Who Cooperated At Least-Three Times

### SQL Solution:

In [ ]:
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
Having COUNT(timestamp) >= 3;

### Pandas Solution:

In [ ]:
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    df = actor_director.groupby(['actor_id', 'director_id']).agg('count').reset_index()
    return df[df['timestamp']>2][['actor_id', 'director_id']]

## 1667. Fix Names In A Table

### SQL Solution:

In [ ]:
SELECT Users.user_id, CONCAT(UPPER(SUBSTR(Users.name,1,1)), LOWER(SUBSTR(Users.name,2))) AS name
FROM Users
ORDER BY Users.user_id ASC;

### Pandas Solution:

In [ ]:
import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    df = users
    df['name'] = df['name'].str.capitalize()
    return df.sort_values(by='user_id')

## 175. Combine Two Tables

### SQL Solution:

In [ ]:
SELECT Person.firstName AS firstName, 
Person.lastName AS lastName, 
Address.city AS city,
Address.state AS state
FROM Person
LEFT JOIN Address
on Person.personId = Address.personId

### Pandas Solution:

In [ ]:
import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    merge_df = pd.merge(person, address, on="personId", how="left")
    return merge_df[['firstName', 'lastName', 'city', 'state']]
  

## 176. Second Highest Salary

### SQL Solution:

In [ ]:
# This method involves finding the maximum salary first and then selecting the maximum salary that is less than the highest.
SELECT MAX(Employee.salary) AS SecondHighestSalary
FROM Employee
WHERE Employee.salary < (
    SELECT MAX(Employee.salary) FROM Employee
    );


### Pandas Solution:

In [ ]:
import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    unique_salary = employee['salary'].drop_duplicates().nlargest(2)
    if (len(unique_salary)) < 2:
        return pd.DataFrame({'SecondHighestSalary': [None]})
    else:
        return pd.DataFrame({'SecondHighestSalary': [unique_salary.iloc[1]]})


## 1327. List The Products Ordered In A Period

### SQL Solution:

In [ ]:
SELECT Products.product_name AS product_name,
SUM(Orders.unit) AS unit
FROM Products
RIGHT JOIN Orders
ON Products.product_id = Orders.product_id
WHERE Orders.order_date LIKE '2020-02-%'
GROUP BY Products.product_id
HAVING unit >= 100;

### Pandas Solution:

In [ ]:
import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # 1. Filter orders to only February 2020
    feb_orders = orders[
        (orders['order_date'] >= '2020-02-01') &
        (orders['order_date'] <= '2020-02-29')
    ]
    
    # 2. Group by product_id and sum the units ordered in that period
    units_by_product = (
        feb_orders
        .groupby('product_id', as_index=False)['unit']
        .sum()
    )
    
    # 3. Keep only products with at least 100 units ordered
    big_orders = units_by_product[units_by_product['unit'] >= 100]
    
    # 4. Join with products table to get product_name
    result = big_orders.merge(products[['product_id', 'product_name']],
                              on='product_id',
                              how='left')
    
    # 5. Return only the required columns
    return result[['product_name', 'unit']]

## 1378. Replace Employee Id With The Unique Identifier

### SQL Solution:

In [ ]:
SELECT EmployeeUNI.unique_id AS unique_id,
Employees.name AS name
FROM Employees
LEFT JOIN EmployeeUNI
ON Employees.id = EmployeeUNI.id

### Pandas Solution:

In [ ]:
import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    result = pd.merge(employees, employee_uni, on='id', how='left')
    return result[['unique_id', 'name']]

## 550. Game Play Analysis Iv

### SQL Solution:

In [ ]:
SELECT
  ROUND(COUNT(DISTINCT player_id) / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM
  Activity
WHERE
  (player_id, DATE_SUB(event_date, INTERVAL 1 DAY))
  IN (
    SELECT player_id, MIN(event_date) AS first_login FROM Activity GROUP BY player_id
  )

### Pandas Solution:

In [ ]:
import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    activity["first"] = activity.groupby("player_id").event_date.transform(min)
    activity_2nd_day = activity.loc[activity["first"] + pd.DateOffset(1) == activity["event_date"]]
    return pd.DataFrame({"fraction":[round(len(activity_2nd_day) / activity.player_id.nunique(),2)]})

## 1075. Project Employees I

### SQL Solution:

In [ ]:
SELECT p.project_id, ROUND(AVG(e.experience_years),2) AS average_years
FROM Project p
LEFT JOIN Employee e
ON p.employee_id = e.employee_id
Group BY p.project_id;

### Pandas Solution:

In [ ]:
import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    merge_table = pd.merge(project, employee, on='employee_id', how='left')
    average_years = merge_table.groupby('project_id')['experience_years'].mean().round(2).reset_index().rename(columns = {'experience_years' : 'average_years'})
    return average_years
    

## 185. Department Top Three Salaries

### SQL Solution:

In [ ]:
SELECT d.name AS 'Department',
       e1.name AS 'Employee',
       e1.salary AS 'Salary'
FROM Employee e1
JOIN Department d
ON e1.departmentId = d.id
WHERE
    3 > (SELECT COUNT(DISTINCT e2.salary)
        FROM Employee e2
        WHERE e2.salary > e1.salary AND e1.departmentId = e2.departmentId
    );

### Pandas Solution:

In [ ]:
import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    employee.  columns = ['id', 'Employee', 'Salary', 'd_id']
    department.columns = ['d_id', 'Department']

    employee['rnk'] = employee.groupby('d_id')[['Salary']
                             ].rank(method='dense', ascending=False)

    return employee[employee.rnk <= 3
                ].merge(department, on='d_id').iloc[:,[5,1,2]]